In [1]:
import mtcnn
import cv2
import numpy as np
from PIL import Image
from keras.models import load_model
from scipy.spatial import distance

In [2]:
def sign_func(image, dict_elements):
    x1, y1, h, w = dict_elements['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + h, y1 + w
    li_coordinates = [x1, y1, h, w]

    gbr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = np.asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = np.asarray(face)

    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std

    face = np.expand_dims(face, axis=0)
    signature = model.predict(face)
    return signature, li_coordinates


def draw_rectangle_base(signature_list, list_coordinates, frame, signature_base):
    dist = distance.cosine(signature_base, signature_list)
    if dist < 0.5:
        cv2.rectangle(frame, tuple(list_coordinates), (0, 255, 0), 2)
    else:
        cv2.rectangle(frame, tuple(list_coordinates), (0, 0, 255), 2)
    return frame

In [3]:
def face_detection(path):
    
    image_base = cv2.imread(path)
    cv2.imshow('img', image_base)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    face = detector.detect_faces(image_base)
    signature_base, li_coordinates_base = sign_func(image_base, face[0])
    
    camera = cv2.VideoCapture(0)

    while True:
        unk, frame = camera.read()
        face = detector.detect_faces(frame)
        if face:
            signature, list_coords = sign_func(frame, face[0])
            frame = draw_rectangle_base(signature, list_coords, frame, signature_base)
        cv2.imshow('videoframe', frame)
        if cv2.waitKey(1) == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

In [5]:
detector = mtcnn.MTCNN()
model = load_model('facenet_keras.h5')

ValueError: bad marshal data (unknown type code)